### March 2023
Newest axon analysis procedures for layer specific project

Cleaned up code from to_lucas.ipynb with detailed comments

April update:
added normalization for tiff stack and excel output
Divide every item by the total number of axons


In [1]:
import os

import pandas as pd

import numpy as np

import SimpleITK as sitk

import warnings

import tkinter.filedialog as fdialog

import random

import matplotlib.pyplot as plt
import Neuron_analysis as na
from Neuron_analysis import *
import skimage
from skimage import io


import re
from tqdm import tqdm

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache


In [2]:
def get_pt_natlas(dspoint_name, to_add):
    '''Read downsampled points after Transformix transformation that are now in Allen Atlas domain, then add the missing y sections that was cropped off previously during the registration step
    Feb 2022
        Modified for viral images where points are indices after trailmap segmentation with probability 
        Since some noise will be picked up as axons and cross probability threshold, it is possible for transformed points to have negative value or outside of the brain
        points with negative values any one dimensions are ignored
    Apr 2022
        Modified to not include atlas name
        Prefer to select atlas directly
    '''
    with open(dspoint_name,'r') as output:
        outputpoint= output.readlines()
    
    all_points=[]

    for lines in tqdm(outputpoint):
        m=re.search("(?:OutputIndexFixed = \[ )([0-9]+ [0-9]+ [0-9]+)", lines)
        if not m:
            #print('negative number in one of the dimension, skipped')
            #print(f'{lines}')
            pass
        else:
            m=m.groups(0)
            this_line= str(m[0]).split(' ')
            mypoints= [int(stuff) for stuff in this_line]
            mypoints[1]= mypoints[1]+to_add
            all_points.append(mypoints)
    
    return all_points

def find_points_id(points):
    
    '''find region id for each coordinate, fix some of noticable errors in allen atlas and potential contribution of misalignment'''

    # for some brains (ie.AL215) because somedownsampled points are located beyond the altas (??), temporarily assign an index of 0
    # this keeps the completeness for later use (ie, propagating region id to original points)
    points_in_atlas=[]
    
    for i in points:
        if i[0]<456 and i[1]<528 and i[2]<320:
            points_in_atlas.append(int(annot_h[i[0], i[1],i[2]]))
        # check if the point index is smaller or equalt to the atlas
        else:
            points_in_atlas.append(0)

    points_in_atlas= np.where(points_in_atlas==484682520, 484682528 , points_in_atlas) 
    points_in_atlas= np.where(points_in_atlas==484682524, 484682528 , points_in_atlas)
    # replace id= 484682520 (optic radiation)  and id= 484682524 (auditory radiation) with 484682258, stc(a subregion of fiber bundle)
    # these are intrinsic issue of the allen atlas, the labels for these regions are wrong
    
    points_in_atlas= np.where(points_in_atlas==382, 466 , points_in_atlas)
    # replace id= 382, hippocampus CA1 with 466, alveus(a subregion of fornix system), to correct for slight misalignment with atlas
    
    return points_in_atlas

def make_tif(all_points, atlas_shape):
    ''' Project downsampled points on to a tif stack, useful for overlaping with brain or template (ie, in imageJ)
    input: downsampled points in a list containing x y z ordinates as int, directory containing it (this is also the output directory) and whether annotation is axon or not (default True)
    example: [[12, 13, 25],
             [13, 14, 25],...]
    
    output: a numpy array with the same dimensions of the template/atlas mhd files with downsampled points only
    each point has a value of the number of occurences (since downsampling combines multiple points as one)
    
    april 2023 update: now outputs an numpy array instead of directly saved as tiff 
    '''
    
    svolume=np.zeros(atlas_shape)
    #columns, rows, planes

    zplanes=[]
    for i in all_points:
        zplanes.append( i[2])
    zplanes=np.unique(zplanes)
    temp=np.zeros(atlas_shape[0:2])
    thepoints=np.asarray(all_points)
    
    real_zplanes= zplanes[zplanes <atlas_shape[2]] # get rid of everything beyond the shape of atlas

    for i in tqdm(real_zplanes):
        index= thepoints[:,2]==i
        uindex,counts=np.unique(thepoints[index],return_counts=True, axis=0)
        for j, lines in enumerate(uindex):
            coord1,coord2=lines[0:2]
            temp[coord1][coord2]= counts[j]
        svolume[:,:,i]=temp #write this in 
        temp=np.zeros(atlas_shape[0:2]) #reset the empty plane after each z


    horizontal_planetmp= np.swapaxes(np.int16(svolume),0,2)
    #for some reason, if just save stuff as tiff, it will save x planes of yz view
    #here we shift the 3rd dimension with the first dimension to obtain xy view
    
    return horizontal_planetmp

def save_tiff(numpystack, outname):
    
    '''save numpy stack as tiff'''
    
    print('Starting to saving tif files..')
    
    io.imsave(outname+'_axons.tif',numpystack)
    
    sum_0=np.sum(numpystack, axis=0)
    sum_1=np.sum(numpystack, axis=1)
    sum_2=np.sum(numpystack, axis=2)
    io.imsave(outname+ '_sum_0.tif',sum_0)
    io.imsave(outname+ '_sum_1.tif',sum_1)
    io.imsave(outname+ '_sum_2.tif',sum_2)

def regions_csv(points_in_atlas,total_axons, out_name):
    
    '''Saves brain region info and counts in excel file.
    Note that coordinates with an brain index of 0 will not get included because it does not have a place in the atlas labels.csv'''

    unique_id, counts = np.unique(points_in_atlas, return_counts=True)
    counts=counts/total_axons #normalize by total number of axon points
    id_withcounts=list(zip(unique_id, counts))

    our_regions=na.atlas_labels.loc[na.atlas_labels['id'].isin (unique_id)]

    new_df= pd.DataFrame(id_withcounts, columns=['id', 'counts'])
    our_regionWcounts=pd.merge(na.atlas_labels, new_df)
    
    our_regionWcounts.to_excel(out_name+'region_with_counts.xlsx',index=None,header=True)

    return our_regionWcounts

def parent_df(df):
    # group dataframe by parent id structure
    grouped_pd=df.groupby(['parent_structure_id'],as_index=False).sum()
    d= {'id': grouped_pd.parent_structure_id.astype(int), 'counts': grouped_pd.counts}
    grouped_pd2= pd.DataFrame(data=d)
    result = pd.merge(grouped_pd2, na.atlas_labels, on=["id"])
    result.sort_values(['counts'], ascending=True, inplace=True)
    # result is the final pd

    return result

def clean_duplicate(df):
    '''This is needed again to account for parent regions that have its subregion incompletely covers the parent areas.
    A painful example is Zona incerta, since it has a depth of 6, after group by parents it will show up twice with different counts where we simply just add the two counts'''
    
    df2=df.groupby(['acronym'],as_index=False, sort=False).sum()
    d= {'acronym': df2.acronym, 'counts': df2.counts}
    grouped_pd2= pd.DataFrame(data=d)
    result = pd.merge(grouped_pd2, na.atlas_labels, on=["acronym"])
    # this merging is required because pd.groupby will drop 'useless columns' such as depth, structure id path and other useful ones!! So we fetch them back here..
    # Probably have better ways of doing it...
    
    return result

In [3]:
mcc = MouseConnectivityCache(resolution=25)
annot, annot_info = mcc.get_annotation_volume()
print('Coronal atlas has shape', annot.shape)
# load allen mouse brain atlas that is in coronal view

annot_h=np.moveaxis(annot, 2, 0)
print('Converted to horizontal atlas with shape', annot_h.shape)
# reslice corontal atlas to horizontal atlas that is consistent with our sample's orientation

atlas_labels=pd.read_csv('D:\Allenbrainatlas\ARA_25_micron_mhd_ccf2017\labels.csv')
# read axon labels

Coronal atlas has shape (528, 320, 456)
Converted to horizontal atlas with shape (456, 528, 320)


In [17]:
points_name=fdialog.askopenfile( title='Select the downsampled transformed axon points').name
# specify the name of transformed axon points after transform_points.py

mouse_name= na.find_mousename(points_name)
# identify mouse id

indir = f'D:\\Viral_stacks\\{mouse_name}\\'
# find directory that contains the cropped template in order to refill the missing y sections

outdir =r'D:\viral_results'
# define folder to store the results

out_name= os.path.join(outdir,mouse_name)
#define a generic output file name for this sample

print(f'now processing {mouse_name}')


now processing AL330


In [18]:
template_name= [i for i in os.listdir(indir) if 'template_' in i]
lead= na.find_crop(template_name[0])
# finds the y section to refil from template name

print (f'refilling the leading {lead} y section for each point')
all_points=get_pt_natlas(points_name,lead)

refilling the leading 67 y section for each point


100%|██████████| 577967/577967 [00:02<00:00, 194909.12it/s]


In [19]:
# now find id for all points first (for future use of associating id to original non-downsample non-transformed points), this is saved as csv file during clean_points function
points_in_atlas_all= find_points_id(all_points) 

d={'coordinates': all_points, 'id': points_in_atlas_all}
new_df =pd.DataFrame(data=d)
# store the corrdinates and their corresponding region id in a pandas dataframe

#all_points_clean=new_df.coordinates.to_list()


In [20]:
real_axons_tmp=new_df[new_df.id!=0]
real_axons= real_axons_tmp.coordinates.tolist()

In [ ]:
tiff_stack= make_tif(real_axons,annot_h.shape)
norm_stack= np.divide(tiff_stack,len(real_axons))

save_tiff(norm_stack, out_name)


100%|██████████| 157/157 [00:01<00:00, 105.85it/s]


Starting to saving tif files..


C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_io.py:141: UserWarning:

D:\viral_results\AL330_axons.tif is a low contrast image



In [ ]:
total_axons=len(real_axons)

left_points= []
right_points= []
for item in all_points:
    if item[0]<227:
        left_points.append(item)
    else:
        right_points.append(item)    

# splits points down the middle in to two lists into left and right hemisphere

In [ ]:
points_in_atlasR= find_points_id(right_points)
regionsR= regions_csv(points_in_atlasR, total_axons, out_name+'_axon_right_')

points_in_atlasL= find_points_id(left_points)
regionsL= regions_csv(points_in_atlasL, total_axons, out_name+'_axon_left_')
# save brain region info and point counts into two separate excel file

In [ ]:
# Generate excel file of regions in parent name (1 graph order up) for left hemisphere

axon_subL=regionsL.sort_values(by=['counts'])
axon_subL.sort_values(by= 'graph_order',axis=0, inplace=True)

needsgroup=axon_subL[axon_subL.depth>6]
noneedsgroup=axon_subL[axon_subL.depth<=6]

parentL= parent_df(needsgroup)
completeL1=noneedsgroup.append(parentL)

completeL=clean_duplicate(completeL1)
# sums up counts from duplicated region, see an example inside function

completeL.sort_values('counts',ascending=False, inplace=True)

completeL.to_excel(f'{out_name}_Lparent.xlsx') 
print('left_parent excel saved')

In [ ]:
# Generate excel file of regions in parent name (1 graph order up) for right hemisphere

if len(regionsR) == 0:
    # still writes an empty file even if there are no axons in the right hemisphere!
    # for completion purposes and later group heatmap
    
    regionsR.to_excel(f'{out_name}_Rparent.xlsx') 
    
else:
    axon_subR=regionsR.sort_values(by=['counts'])
    axon_subR.sort_values(by= 'graph_order',axis=0, inplace=True)

    needsgroupR=axon_subR[axon_subR.depth>6]
    noneedsgroupR=axon_subR[axon_subR.depth<=6]

    parentR= parent_df(needsgroupR)

    completeR1=noneedsgroupR.append(parentR)

    completeR=clean_duplicate(completeR1)

    completeR.sort_values('counts',ascending=False, inplace=True)

    completeR.to_excel(f'{out_name}_Rparent.xlsx') 
print('right_parent excel saved')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def arrange_parent_subregion(parentdf,subregiondf):
    print('Re-arranged subregion order based on parent region that has the greatest number of axons...')
    
    new_order=parentdf.id.to_numpy()
    
    old_order= subregiondf.parent_structure_id.to_numpy()   
    
    #print('Old parent region id order is: ', old_order)
    #print('New parent region id order is: ', new_order)
    
    
    new_array= np.zeros_like(old_order)
    for i, j in enumerate(new_order):
        new_array[old_order==j]=i
        
    #print('Re-arranged subregion order based on parent region that has the greatest number of axon: ', new_array)
    
    subregiondf['new_order']= new_array
    subregiondf.sort_values('new_order', inplace=True)
    
    subregiondf.sort_values(by=['new_order', 'graph_order'], ascending=[True, True] ,inplace=True)

    return subregiondf

def plot_hist(pd_axonL,pd_axonR, out_name):
    ''' 
    Plot horizontal histogram of all points and ending points of axons and dendrites
    Input: pandas dataframe of axon, pandas dataframe of dendrite, mousename
    '''

    fig = make_subplots(
        shared_yaxes=True,
        rows=2, cols=1,
        row_heights=[0.6, 0.5],
        row_titles=['Left axons', 'Right axons', 'Dendrites']
    )
    
    fig.add_trace(
        go.Bar(
        y=pd_axonL['acronym'], 
        #x=pd_axonL['intensity'],
        x=pd_axonL['counts']/1000, # units now in milimeters
        marker_color='red', #for future, pd_axon['region_id'],
        name='',
        text=pd_axonL['name'],
        hovertemplate=
            '<i>%{x}</i>, '+
            '<b>%{text}</b>',
        orientation='h'),
        row=1,col=1
    )
    
    fig.add_trace(
        go.Bar(
        y=pd_axonR['acronym'], 
        #x=pd_axonR['intensity'],
        x=pd_axonR['counts']/1000, # units now in milimeters
        marker_color='magenta', #for future, pd_axon['region_id'],
        name='',
        text=pd_axonR['name'],
        hovertemplate=
            '<i>%{x}</i>, '+
            '<b>%{text}</b>',
        orientation='h'),
        row=2,col=1
    )
    

   
    fig.update_layout(yaxis={'categoryorder':'trace'}, 
                      width=2000,
                      height=1000, # 1500 for AL066 since too many items
                      showlegend= False,
                      paper_bgcolor='rgba(0,0,0,0)', # transparent background
                      plot_bgcolor='rgba(0,0,0,0)' # transparent background
                     )
    
    fig.update_xaxes(gridcolor='gold')
    
    fig.show()

    fig.write_image(f"{out_name}.svg")
    fig.write_html(f"{out_name}.html")

In [ ]:
print('For left hemisphere')
new_axonL= arrange_parent_subregion(completeL,axon_subL)


print('For right hemisphere')
if len(regionsR) == 0:
    new_axonR= regionsR
    print('There are no axons in the right hemisphere')
else:
    new_axonR= arrange_parent_subregion(completeR,axon_subR)

In [ ]:
plot_hist(new_axonL,new_axonR,out_name)